In [11]:
import pandas as pd
import numpy as np

# Load the data from the csv into a dataframe
df = pd.read_csv('word_embeddings.csv')
# Drop unnecessary column
df.drop('Unnamed: 0', axis='columns', inplace=True)

In [14]:
from gensim.models import KeyedVectors

filename = 'word2vecSmall.bin.gz'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

# Add word embeddings to dataframe
df['embedding']

In [15]:
# These are the words we want to test our model with - feel free to change these!
blacklist = ['nice', 'mean', 'bad', 'good', 'sad', 'happy', 'fantastic', 'terrible']

In [16]:
# This is the data we are testing our model on (contains words in blacklist)
test_df = df[df['word'].str.lower().str.strip().isin(blacklist)].copy()
test_df.head()

,word,score
16,good,0.500
106,bad,-0.875
164,happy,0.875
649,Good,0.500
717,fantastic,0.375


In [17]:
# This is the data we are training our model on (words NOT in blacklist)
train_df = df[~df['word'].str.lower().str.strip().isin(blacklist)].copy()
train_df.head()

,word,score
0,is,0.125
1,not,-0.625
2,will,0.125
3,an,-0.125
4,had,0.250


In [6]:
# Ensures input to LinearRegression model is in acceptable format
X_train = np.array(list(train_df['embedding'].values))
Y_train = np.array(list(train_df['score'].values))

X_test = np.array(list(test_df['embedding'].values))
Y_test = np.array(list(test_df['score'].values))

In [7]:
from sklearn.linear_model import LinearRegression

classifier = LinearRegression()

classifier.fit(X_train, Y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [8]:
print("Score:", classifier.score(X_test, Y_test))

Score: 0.5823369832683984


In [9]:
# Here's an example of output from the model - 'happy' should have a higher sentiment score than 'sad'
print(classifier.predict([np.array(model['happy'])])[0])
print(classifier.predict([np.array(model['sad'])])[0])

NameError: name 'model' is not defined

In [ ]:
# This gets the shape of coefficients of the LinearRegression model, which we can see is the same
# as the length of the embedding input
classifier.coef_.shape

In [ ]:
# This is the intercept of the LinearRegression model
classifier.intercept_

In [ ]:
test_df['predicted_score'] = test_df['embedding'].apply(lambda x: classifier.predict(x.reshape(1, -1))[0])

In [ ]:
test_df.head()

In [ ]:
# A bar plot showing what our model thinks the sentiments of our test words are
test_df.plot(x='word',y='predicted_score',kind='bar')

In [ ]:
# Here's what they actually are - should look similar to previous plot!
test_df.plot(x='word',y='score',kind='bar')